<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/air_pollution_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

First, I used the data about air_polution data from 
Ministry of Environment

In [ ]:
import pandas as pd
import json
import tensorflow as tf
import tensorflow.keras as K
import numpy as np

I got the tip from how to read the file with utf-8 encoding from [here](https://teddylee777.github.io/pandas/%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%95%9C%EA%B8%80%EA%B9%A8%EC%A7%90%ED%98%84%EC%83%81-%ED%95%B4%EA%B2%B0%EB%B0%A9%EB%B2%95)

In [ ]:
def read_data():
  data = pd.read_csv("/content/drive/My Drive/Datasets/air_2019.csv", encoding='utf-8')
  return data

In [ ]:
data = read_data()

In [ ]:
data

,측정일자,권역코드,권역명,측정소코드,측정소명,미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),초미세먼지(㎍/㎥)
0,20191231,100,도심권,111121,중구,26,18,0.023,0.012,0.4,0.002
1,20191231,100,도심권,111123,종로구,27,19,0.021,0.015,0.4,0.003
2,20191231,100,도심권,111131,용산구,14,15,0.020,0.015,0.4,0.003
3,20191231,101,서북권,111181,은평구,24,13,0.024,0.010,0.4,0.003
4,20191231,101,서북권,111191,서대문구,23,9,0.023,0.013,0.5,0.002
...,...,...,...,...,...,...,...,...,...,...,...
8891,20190101,103,서남권,111301,양천구,46,29,0.014,0.033,0.6,0.002
8892,20190101,104,동남권,111261,강남구,37,31,0.012,0.028,0.6,0.005
8893,20190101,104,동남권,111262,서초구,37,20,0.015,0.032,0.4,0.004
8894,20190101,104,동남권,111273,송파구,44,28,0.011,0.042,0.8,0.004


Filtering data

In [ ]:
location = data['측정소명'] == "중구"

dust_location = data[location]
new_list = pd.DataFrame(dust_location['미세먼지(㎍/㎥)'])
new_list

,미세먼지(㎍/㎥)
0,26
25,36
50,46
75,43
100,34
...,...
8771,63
8796,55
8821,35
8846,31


In [ ]:
def read_dust_data():
  data = pd.read_excel("/content/drive/My Drive/Datasets/APIS/APIS_data.xlsx", encoding='utf-8')
  return data

In [ ]:
data = read_dust_data()

In [ ]:
data

,1.0ug/m3,2.5ug/m3
0,21,29
1,21,30
2,21,30
3,22,32
4,22,32
...,...,...
3327,25,37
3328,25,36
3329,26,37
3330,26,38


# 공공데이터 받기

In [17]:
import urllib
from urllib.parse import urlencode, quote_plus, unquote
from urllib.request import urlopen
import requests

decode_key = unquote('cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D')
#-*- coding:utf-8 -*-
# url = 'http://apis.data.go.kr/1360000/LivingWthrIdxService/getDiscomfortIdx' # 불쾌지수조회
url = 'http://apis.data.go.kr/1360000/LivingWthrIdxService/getHeatFeelingIdx' 
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : decode_key, quote_plus('pageNo') : '1', quote_plus('dataType') : 'XML', quote_plus('areaNo') : '1100000000', quote_plus('time') : '2020072506',quote_plus('requestCode') : 'A20'})

request = requests.get(url + queryParams)
# response = urlopen(request)
print(request.text)

<?xml version="1.0" encoding="UTF-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><code>A20</code><areaNo>1100000000</areaNo><date>2020072506</date><h3>23</h3><h6>25</h6><h9>26</h9><h12>25</h12><h15>23</h15><h18>22</h18><h21>22</h21><h24>21</h24><h27>24</h27><h30>26</h30><h33>26</h33><h36>26</h36><h39>25</h39><h42>23</h42><h45>23</h45><h48>22</h48><h51>23</h51><h54>24</h54><h57>25</h57><h60>24</h60><h63>24</h63><h66>23</h66></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>1</totalCount></body></response>



In [ ]:
http://apis.data.go.kr/1360000/LivingWthrIdxService/getSenTaIdx?